In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_137572\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_00360s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-671.267311,-90.259375,1,3,2091.391496,3823.292859,0.345128,0.321786,1,-671.267311,"-1773,-2",-27.173147,-895.245495,-5.995013,-74.191610,0.762949
1,2,1,1,NaN,-425.041608,-77.950447,0,2,2090.529471,3821.000000,0.499131,0.000061,1,-425.041608,"-1773,-1",-26.737013,-894.089744,-5.826496,-73.992142,0.508633
2,3,1,1,NaN,-687.554293,-83.046325,0,3,2086.313863,3820.966777,0.328141,0.574388,1,-687.554293,-1773,-24.611071,-894.075595,-5.361884,-73.992423,0.762949
3,5,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1778,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
4,6,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1783,105.836138,-888.242242,21.967109,-72.768028,0.762949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485969,636938,97,8,NaN,157.588062,32.583864,0,4,211.751755,405.715430,0.477116,0.462478,0,NaN,None,-180.731023,607.081885,-13.333716,34.991202,65.266592
1485970,636939,97,8,NaN,117.236930,28.223411,0,2,390.489913,415.000000,0.499898,0.000000,0,NaN,None,541.261585,644.585859,47.186604,39.436700,32.633296
1485971,636940,97,8,NaN,138.886764,29.665144,0,2,129.535331,431.000000,0.498750,0.000008,0,NaN,None,-512.834981,709.216052,-49.560602,45.126665,32.633296
1485972,636941,97,8,NaN,94.020804,24.698222,0,2,172.484928,440.000000,0.499773,0.000000,1,NaN,"24548,520146",-339.344935,745.570536,-31.705606,47.488884,32.633296


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 489731/489731 [2:00:08<00:00, 67.94it/s]  


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_00360s.pnds')